In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)

In [3]:
# Load Dataset
file_path = "districtwise-crime-against-sts-2017-onwards.csv"
df = pd.read_csv(file_path)
print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

Dataset shape: (5468, 45)
Columns: ['id', 'year', 'state_name', 'state_code', 'district_name', 'district_code', 'registration_circles', 'murder', 'atmpt_commit_murder', 'simple_hurt', 'grievous_hurt', 'acid_attack', 'atmpt_acid_attack', 'other_grievous_hurt', 'assault_on_adult_women', 'sexual_harassment', 'atmpt_disrobe', 'voyeurism', 'stalking', 'assault_of_children_pocso', 'inslt_women_modesty', 'missing_children_kidnpd', 'other_kidnp_abduc', 'kidnp_begging', 'kidnp_murder', 'kidnapping_for_ransom', 'kidnp_marriage', 'procur_minor_girls', 'kidnp_abduc_sec_365_to_369', 'rape_of_women', 'rape_of_children', 'atmpt_rape', 'rioting', 'robbery', 'dacoity_with_murder', 'dacoity', 'arson', 'criminal_intimidation', 'other_ipc_crimes', 'intimitate_humiliate', 'occupy_land_blngs_sts', 'prvnt_obstruct_usage_pub_passage', 'frc_leave_place_res_soc_boycott', 'other_offences', 'protection_of_civil_rights']


In [9]:
year_col = None
for c in df.columns:
    if 'year' in c:
        year_col = c
        break

In [ ]:
if year_col:
    df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
else:
    print("⚠️ No 'Year' column found — creating a dummy year index.")
    df['year'] = np.arange(len(df))
    year_col = 'year'

df = df.dropna(how='all')

num_cols = df.select_dtypes(include=np.number).columns
df['total_offenses'] = df[num_cols].sum(axis=1)

district_col = None
for c in df.columns:
    if 'district' in c:
        district_col = c
        break

if district_col:
    df['prev_year_total'] = (
        df.sort_values([district_col, year_col])
        .groupby(district_col)['total_offenses']
        .shift(1)
    )
else:
    df['prev_year_total'] = df['total_offenses'].shift(1)


In [5]:

# Fill NaN values
df = df.fillna(0)

In [11]:
# Feature Selection
features = [year_col, 'prev_year_total']

for c in df.columns:
    if 'state' in c and 'code' in c:
        features.append(c)
        break

print("Using features:", features)

X = df[features]
y_reg = df['total_offenses']
y_clf = (df['total_offenses'] > df['total_offenses'].median()).astype(int)

Using features: ['year', 'prev_year_total', 'state_code']


In [12]:
X_train, X_test, y_train_reg, y_test_reg, y_train_clf, y_test_clf = train_test_split(
    X, y_reg, y_clf, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Training Classification Models
clf_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest Classifier': RandomForestClassifier(n_estimators=100, random_state=42)
}

clf_results = {}
for name, model in clf_models.items():
    model.fit(X_train, y_train_clf)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else y_pred

    clf_results[name] = {
        'Accuracy': accuracy_score(y_test_clf, y_pred),
        'Precision': precision_score(y_test_clf, y_pred),
        'Recall': recall_score(y_test_clf, y_pred),
        'F1': f1_score(y_test_clf, y_pred),
        'ROC_AUC': roc_auc_score(y_test_clf, y_prob)
    }


In [ ]:
# Training Regression Models
reg_models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=42)
}

reg_results = {}
for name, model in reg_models.items():
    model.fit(X_train, y_train_reg)
    y_pred = model.predict(X_test)

    reg_results[name] = {
        'MSE': mean_squared_error(y_test_reg, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_reg, y_pred)),
        'MAE': mean_absolute_error(y_test_reg, y_pred),
        'R²': r2_score(y_test_reg, y_pred)
    }

In [ ]:
print("\n CLASSIFICATION RESULTS:")
for model, metrics in clf_results.items():
    print(f"\n{model}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

print("\n REGRESSION RESULTS:")
for model, metrics in reg_results.items():
    print(f"\n{model}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


📊 CLASSIFICATION RESULTS:

Logistic Regression:
  Accuracy: 0.8601
  Precision: 0.8644
  Recall: 0.8500
  F1: 0.8571
  ROC_AUC: 0.9040

Random Forest Classifier:
  Accuracy: 0.9662
  Precision: 0.9736
  Recall: 0.9574
  F1: 0.9655
  ROC_AUC: 0.9847

📈 REGRESSION RESULTS:

Linear Regression:
  MSE: 20354511.1280
  RMSE: 4511.5974
  MAE: 3046.7607
  R²: 0.4519

Random Forest Regressor:
  MSE: 8378823.7703
  RMSE: 2894.6198
  MAE: 1001.4580
  R²: 0.7744


In [16]:
#comparing models
best_clf = max(clf_results.items(), key=lambda x: x[1]['F1'])
best_reg = max(reg_results.items(), key=lambda x: x[1]['R²'])


In [17]:
print(f"Best Classifier: {best_clf[0]} (F1: {best_clf[1]['F1']:.3f}, ROC_AUC: {best_clf[1]['ROC_AUC']:.3f})")
print(f"Best Regressor: {best_reg[0]} (R²: {best_reg[1]['R²']:.3f}, RMSE: {best_reg[1]['RMSE']:.3f})")

Best Classifier: Random Forest Classifier (F1: 0.965, ROC_AUC: 0.985)
Best Regressor: Random Forest Regressor (R²: 0.774, RMSE: 2894.620)
